In [ ]:
import pandas as pd
import re
import os
from unicodedata import normalize
from bs4 import BeautifulSoup

In [ ]:
input_1 = '5000 zł'
input_2 = '4000 do 5000 PLN'
input_3 = '5.000 - 6.000 zł'
input_4 = 'from 10 000 to 15 000 PLN'


In [ ]:
def parse_salary(salary) -> dict:
    salary = normalize('NFKD', salary) #znaki diakrytycznymi/ specjalne - zeby czytalo
    bounds = re.findall('[0-9]+', salary.replace(' ', ''))
    low_bound = bounds[0]
    high_bound = bounds[1] if len(bounds) > 1 else bounds[0]
    currency = salary.split()[-1]

    return {'low': low_bound,
            'high': high_bound,
            'currency': currency}

In [ ]:
parse_salary(input_2)

In [ ]:
def parse_city(city) -> dict:
    if re.search('(Zdalnie)', city):
         _city = "Zdalna",
    else:
        _city = city

    return {'city': _city}

In [ ]:
parse_city("Zdalnie")

In [ ]:
# A to dlatego że lista nie ma takiej metody. Dopiero gdy iterujemy przez elemnty w liscie ktore są obiektami BeautifulSoup
# to mozemy wywaolac find() w funkcji parse_job

def parse_jobs(jobs) -> list:
    results = []

    for job in jobs:
        results.append(parse_job(job))

    return results

In [ ]:
def parse_job(job) -> dict:
#przetwarzamy informacje o pojedynczej ofercie pracy
#salary & lokalizacja - pobieramy to z TAGOW HTML -> i na tym aplikujemy nasze funkcje
    job_info = job.find('nfj-posting-item-title', class_='align-items-lg-center')
    # print(job_info)

    salary = job_info.find_all('span')[0].text.strip() #strip usuwa puste spacje
    salary_data = parse_salary(salary)
    # print(salary)

    location = job.find('nfj-posting-item-city')
    if location is None:
        location = 'Not listed'
    else:
        location = location.find('span')
        location = location.text.strip()


    location = parse_city(location)
    # print(location)

    name = job.find('h3', class_='posting-title__position').text.strip()
    # print(name)
#select_one -> selektory CSS
    # company = job.select_one("div[_ngcontent-serverapp-c302]>span[data-cy='company name on the job offer listing']").text.strip()

##POPRAW COMPANY!!!!
    company = job.find('span', class_='d-block')#.text.replace('w', '').strip()
    # company = job.select_one("div[_ngcontent-serverapp-c302]>span[data-cy='company name on the job offer listing']")
    if company is not None:
        company = company.text.strip()
    else:
        company = "Unknown"

    # print(company)
    technology = job.select_one("span[data-cy='category name on the job offer listing']")

    #A gdyby zamiast powyższego select_one użyć find to będzie wyglądać polecenie jak poniżej:
    # job.find('span', attrs={"data-cy": "category name on the job offer listing"})

    if technology:
        technology = technology.text.strip()
    else:
        technology = 'N/A'

    return {
        'location': location,
        'salary': salary_data,
        'name': name,
        'company': company,
        'technology': technology
    }

In [ ]:
def get_data(page) -> list:
    results = []
# Wywołanie x.parent zwraca rodzica (bezpośredniego nadrzędnego elementu) dla tego elementu.
    jobs = [x.parent for x in page.find_all('div', class_='posting-image')]
    page_data = parse_jobs(jobs)
    results += page_data

    return results

In [ ]:
# przyklad
with open('/Users/katarzynahewelt/Desktop/Materiały na zjazd/zjazd_5_po_zajeciach/warsztat_oraz_struktura_plikow_cookiecutter/data/raw/data scientist_1_20230214_001527.html', 'r', encoding='utf-8') as f:
    page_html = f.read()

page = BeautifulSoup(page_html, 'html.parser')

results = get_data(page)
print(results)
